This .ipynb file contains the code for the counts of Ffaudulent and not fraudulent transactions, range of the amount spent given that a transaction was fraud, the logistic regreeion model, naivebayes model, and gradient (XGBoost) model.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('/Users/sandersander/Documents/CS 548/Project 2/credit_card_transactions.csv')

df.head()
print(df.columns)

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud', 'merch_zipcode'],
      dtype='object')


In [ ]:
# Group by the 'is_fraud' column and count the number of transactions for fraud and not fraud
fraud_counts = df['is_fraud'].value_counts()

# Plot the fraud and not fraud counts
plt.figure(figsize=(6, 4))
fraud_counts.plot(kind='bar', color=['green', 'red'])
plt.title('Total Count of Fraud and Not Fraud Transactions')
plt.xlabel('Is Fraud')
plt.ylabel('Number of Transactions')
plt.xticks([0, 1], ['Not Fraud', 'Fraud'], rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Code to display the range of the amount spent given that a transaction was fraud
fraud_df = df[df['is_fraud'] == 1]

# Create bins for transaction amounts in $100 increments
bins = range(0, int(fraud_df['amt'].max()) + 100, 100)
fraud_df['amt_range'] = pd.cut(fraud_df['amt'], bins)

# Count the number of fraudulent transactions in each range
fraud_amount_counts = fraud_df['amt_range'].value_counts().sort_index()

# Plot the results
plt.figure(figsize=(10, 6))
fraud_amount_counts.plot(kind='bar', color='red')
plt.title('Fraudulent Transactions by Amount (in $100 Increments)')
plt.xlabel('Transaction Amount Range ($)')
plt.ylabel('Number of Fraudulent Transactions')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [5]:
# Columns to remove to make the classifier models
columns_to_remove = [
    'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
    'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
    'merch_lat', 'merch_long', 'merch_zipcode', 'Unnamed: 0'
]

# Remove specified columns
df_cleaned = df.drop(columns=columns_to_remove)

# Display the cleaned DataFrame
print(df_cleaned.columns)

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'is_fraud'],
      dtype='object')


Naive Bayes classifier code

In [6]:
!pip install pandas scikit-learn numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encoding categorical features
df['merchant'] = LabelEncoder().fit_transform(df['merchant'])
df['category'] = LabelEncoder().fit_transform(df['category'])

# Features and target variable
X = df[['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt']]
y = df['is_fraud']

# Convert 'trans_date_trans_time' to numerical (if needed)
X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time']).astype(int) / 10**9  # Convert to Unix timestamp

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


/var/folders/wz/m351thxn2fxf9m9qhvjc__nh0000gn/T/ipykernel_12006/2441686278.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time']).astype(int) / 10**9  # Convert to Unix timestamp


In [7]:
from sklearn.naive_bayes import GaussianNB

# Create a Naive Bayes classifier
model = GaussianNB()

# Train the model
model.fit(X_train, y_train)


GaussianNB()

In [8]:
# Make predictions
y_pred = model.predict(X_test)


In [9]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.99
Confusion Matrix:
[[257815      0]
 [  1520      0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    257815
           1       0.00      0.00      0.00      1520

    accuracy                           0.99    259335
   macro avg       0.50      0.50      0.50    259335
weighted avg       0.99      0.99      0.99    259335



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
# Feature Engineering
# Convert 'amt' into categorical feature based on predefined ranges
def categorize_amount(amount):
    if amount < 50:
        return 'Low'
    elif 50 <= amount < 200:
        return 'Medium'
    else:
        return 'High'

X['amount_category'] = X['amt'].apply(categorize_amount)

# Extract hour from 'trans_date_trans_time' and add it as a new feature
X['transaction_hour'] = pd.to_datetime(X['trans_date_trans_time'], unit='s').dt.hour

# Dropping the original 'amt' column for now (you can keep it if you want)
X = X.drop(columns=['amt'])

# Encode the new categorical feature
X['amount_category'] = LabelEncoder().fit_transform(X['amount_category'])


/var/folders/wz/m351thxn2fxf9m9qhvjc__nh0000gn/T/ipykernel_12006/1988257201.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['amount_category'] = X['amt'].apply(categorize_amount)
/var/folders/wz/m351thxn2fxf9m9qhvjc__nh0000gn/T/ipykernel_12006/1988257201.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['transaction_hour'] = pd.to_datetime(X['trans_date_trans_time'], unit='s').dt.hour


In [11]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cross_val_scores = cross_val_score(model, X, y, cv=5)  # 5-fold cross-validation
print(f'Cross-validation scores: {cross_val_scores}')
print(f'Mean cross-validation score: {cross_val_scores.mean():.2f}')


Cross-validation scores: [0.99421212 0.99421212 0.99421212 0.99421212 0.99420826]
Mean cross-validation score: 0.99


In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Encoding categorical features
df['merchant'] = LabelEncoder().fit_transform(df['merchant'])
df['category'] = LabelEncoder().fit_transform(df['category'])

# Features and target variable
X = df[['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt']]
y = df['is_fraud']

# Convert 'trans_date_trans_time' to numerical (if needed)
X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time']).astype(int) / 10**9  # Convert to Unix timestamp

# Feature Engineering
# Convert 'amt' into categorical feature based on predefined ranges
def categorize_amount(amount):
    if amount < 50:
        return 'Low'
    elif 50 <= amount < 200:
        return 'Medium'
    else:
        return 'High'

X['amount_category'] = X['amt'].apply(categorize_amount)

# Extract hour from 'trans_date_trans_time' and add it as a new feature
X['transaction_hour'] = pd.to_datetime(X['trans_date_trans_time'], unit='s').dt.hour

# Dropping the original 'amt' column for now (you can keep it if you want)
X = X.drop(columns=['amt'])

# Encode the new categorical feature
X['amount_category'] = LabelEncoder().fit_transform(X['amount_category'])

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train Naive Bayes classifier
model = GaussianNB()
model.fit(X_train, y_train)

# Perform cross-validation
cross_val_scores = cross_val_score(model, X, y, cv=5)  # 5-fold cross-validation
print(f'Cross-validation scores: {cross_val_scores}')
print(f'Mean cross-validation score: {cross_val_scores.mean():.2f}')

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy of the model: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)





/var/folders/wz/m351thxn2fxf9m9qhvjc__nh0000gn/T/ipykernel_12006/2259630957.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time']).astype(int) / 10**9  # Convert to Unix timestamp
/var/folders/wz/m351thxn2fxf9m9qhvjc__nh0000gn/T/ipykernel_12006/2259630957.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['amount_category'] = X['amt'].apply(categorize_amount)
/var/folders/wz/m351thxn2fxf9m9qhvjc__nh0000gn/T/ipykernel_12006/2259630957.

Cross-validation scores: [0.99421212 0.99421212 0.99421212 0.99421212 0.99420826]
Mean cross-validation score: 0.99


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy of the model: 0.99
Confusion Matrix:
[[257815      0]
 [  1520      0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    257815
           1       0.00      0.00      0.00      1520

    accuracy                           0.99    259335
   macro avg       0.50      0.50      0.50    259335
weighted avg       0.99      0.99      0.99    259335



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Code for XGBoost model

In [27]:
!pip install xgboost
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


# Encoding categorical features
df['merchant'] = LabelEncoder().fit_transform(df['merchant'])
df['category'] = LabelEncoder().fit_transform(df['category'])

# Features and target variable
X = df[['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt']]
y = df['is_fraud']

# Convert 'trans_date_trans_time' to numerical (if needed)
X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time']).astype(int) / 10**9  # Convert to Unix timestamp

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the XGBoost classifier
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy of the model: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)



[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


/var/folders/wz/m351thxn2fxf9m9qhvjc__nh0000gn/T/ipykernel_12006/343580105.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time']).astype(int) / 10**9  # Convert to Unix timestamp
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:52:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy of the model: 1.00
Confusion Matrix:
[[257649    166]
 [   555    965]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    257815
           1       0.85      0.63      0.73      1520

    accuracy                           1.00    259335
   macro avg       0.93      0.82      0.86    259335
weighted avg       1.00      1.00      1.00    259335



In [29]:

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


# Encoding categorical features merchant and category
df['merchant'] = LabelEncoder().fit_transform(df['merchant'])
df['category'] = LabelEncoder().fit_transform(df['category'])

# Features and target variable needed
X = df[['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt']]
y = df['is_fraud']

X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time']).astype(int) / 10**9

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the XGBoost classifier with class weights
model = XGBClassifier(scale_pos_weight=len(y_train) / (2 * sum(y_train)), use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Evaluate the model using cross-validation
cv_scores = cross_val_score(model, X_train, y_train, cv=5)  # 5-fold cross-validation
print(f'Cross-Validation Scores: {cv_scores}')
print(f'Mean CV Accuracy: {cv_scores.mean():.2f}')


y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy of the model: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)


/var/folders/wz/m351thxn2fxf9m9qhvjc__nh0000gn/T/ipykernel_12006/539637460.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time']).astype(int) / 10**9  # Convert to Unix timestamp
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [20:11:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [20:11:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not u

Cross-Validation Scores: [0.98728961 0.98793067 0.98776679 0.98833555 0.98692328]
Mean CV Accuracy: 0.99
Accuracy of the model: 0.99
Confusion Matrix:
[[254715   3100]
 [   110   1410]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    257815
           1       0.31      0.93      0.47      1520

    accuracy                           0.99    259335
   macro avg       0.66      0.96      0.73    259335
weighted avg       1.00      0.99      0.99    259335



Code for logistic regression

In [30]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


# Encoding categorical features
df['merchant'] = LabelEncoder().fit_transform(df['merchant'])
df['category'] = LabelEncoder().fit_transform(df['category'])

# Features and target variable
X = df[['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt']]
y = df['is_fraud']

X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time']).astype(int) / 10**9  # Convert to Unix timestamp

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create, train, predict the Logistic Regression model
model = LogisticRegression(max_iter=1000)  # Increase max_iter for convergence if needed
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy of the model: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)


/var/folders/wz/m351thxn2fxf9m9qhvjc__nh0000gn/T/ipykernel_12006/2086092675.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time']).astype(int) / 10**9  # Convert to Unix timestamp
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

Accuracy of the model: 0.99
Confusion Matrix:
[[257815      0]
 [  1520      0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    257815
           1       0.00      0.00      0.00      1520

    accuracy                           0.99    259335
   macro avg       0.50      0.50      0.50    259335
weighted avg       0.99      0.99      0.99    259335



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [31]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE


# Encoding categorical features
df['merchant'] = LabelEncoder().fit_transform(df['merchant'])
df['category'] = LabelEncoder().fit_transform(df['category'])

# Features and target variable
X = df[['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt']]
y = df['is_fraud']

# Convert 'trans_date_trans_time' to numerical (if needed)
X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time']).astype(int) / 10**9  # Convert to Unix timestamp

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SMOTE technique
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Standardizing the features
scaler = StandardScaler()
X_resampled = scaler.fit_transform(X_resampled)
X_test = scaler.transform(X_test)

# Create, train, predict the Logistic Regression model
model = LogisticRegression(C=0.5, penalty='l2', solver='liblinear', max_iter=1000)
model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy of the model: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)


/var/folders/wz/m351thxn2fxf9m9qhvjc__nh0000gn/T/ipykernel_12006/1650099896.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time']).astype(int) / 10**9  # Convert to Unix timestamp


Accuracy of the model: 0.95
Confusion Matrix:
[[245370  12445]
 [   340   1180]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97    257815
           1       0.09      0.78      0.16      1520

    accuracy                           0.95    259335
   macro avg       0.54      0.86      0.57    259335
weighted avg       0.99      0.95      0.97    259335



Here I am citing the ChatGPT resourse with prompts and outputs.

Prompt:How can you Implement SMOTE in a logistic regression model, what would be the code to do that
Output:
from imblearn.over_sampling import SMOTE

# Create an instance of SMOTE
smote = SMOTE(random_state=42)

# Fit and resample the training data
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Verify the new class distribution
print(pd.Series(y_resampled).value_counts())

Prompt: How do I incorperate class weights in the XGBoost model? What would the code be?
Output:
# Create and train the XGBoost classifier with class weights
model = XGBClassifier(scale_pos_weight=len(y_train) / (2 * sum(y_train)), use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

Prompt: How do you convert 'amt' into a categorical feature bae on predefind ranges? whats would the code be?
Output:
# Feature Engineering
# Convert 'amt' into categorical feature based on predefined ranges
def categorize_amount(amount):
    if amount < 50:
        return 'Low'
    elif 50 <= amount < 200:
        return 'Medium'
    else:
        return 'High'

X['amount_category'] = X['amt'].apply(categorize_amount)

Prompt: How would I code a Naive Bayes classifier model to predict fraud transaction with this dataset? What would the code look like.
Output: 
# Load your dataset
df = pd.read_csv('your_dataset.csv')  # Adjust this line based on your data file

# Encoding categorical features
df['merchant'] = LabelEncoder().fit_transform(df['merchant'])
df['category'] = LabelEncoder().fit_transform(df['category'])

# Features and target variable
X = df[['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt']]
y = df['is_fraud']

# Convert 'trans_date_trans_time' to numerical (if needed)
X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time']).astype(int) / 10**9  # Convert to Unix timestamp

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train Naive Bayes classifier
model = GaussianNB()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)